In [1]:
!which python

/home/tonatiu/Analysis/.venv/bin/python


In [17]:
#requirement.txt
!pip install pandas
!pip install openpyxl
!pip install streamlit
!pip install plotly
!pip install matplotlib

  Using cached matplotlib-3.9.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached contourpy-1.3.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 kB 5.8 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.7-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.3 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.9.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.3 MB)
Using cached contourpy-1.3.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (323 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 11.3 MB/s eta 0:00:00m eta 0:00:010:01:01
Using cached kiwisolver-1.4.7-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.5 MB)
Using cached pyparsin

In [18]:
import pandas as pd
import streamlit
import plotly.express as px
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio

# Database

In [21]:
# Read database
df = pd.read_excel("./Tabela_Transformada_GE_Tales.xlsx")

df.fillna("", inplace=True)

df.head(5)

,Colaborador,Marco zero,A1.1,A1.2,A2.1,A2.2,B1.1,B1.2,B2.1,B2.2,B2+.1,B2+.2,C1.1,C1.2
0,Adriano Apolinario,09/2023,07/2024,NaN,NaN,12/2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Adriano Carvalho,NaN,NaN,NaN,04/2024,NaN,NaN,NaN,06/2025,NaN,NaN,NaN,NaN,NaN
2,Aguinaldo Jesus,NaN,NaN,NaN,NaN,09/2023,07/2024,NaN,NaN,12/2025,NaN,NaN,NaN,NaN
3,Alessandro Camargo Lopes,NaN,NaN,NaN,NaN,NaN,03/2023,NaN,07/2024,NaN,06/2025,NaN,NaN,NaN
4,Alex Sales,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/2023,NaN,07/2024,NaN,06/2025,NaN


# TeamMap

## Distribuição de pessoas por nivel de proficiência

In [34]:
# Set the renderer to a compatible interactive one
pio.renderers.default = "iframe"  

# Load data
file_path = 'Planilha_inicial.xlsx'
sheet_name = 'Consolidado'
data = pd.read_excel(file_path, sheet_name=sheet_name)

# Rename the column for easier access
data.rename(columns={'Avaliação de proficiencia (classificação': 'Proficiency'}, inplace=True)

# Replace 'Ground zero' with 'Marco zero'
data['Proficiency'] = data['Proficiency'].replace('Ground zero', 'Marco zero')

# Define the order of CEFR levels
cefr_levels = [
    "Marco zero", "A1.1", "A1.2", "A2.1", "A2.2", 
    "B1.1", "B1.2", "B2.1", "B2.2", "B2+.1", 
    "B2+.2", "C1.1", "C1.2", "C1.3", "C2"
]

# Count occurrences of each CEFR level
proficiency_counts = data['Proficiency'].value_counts().reindex(cefr_levels, fill_value=0)

# Calculate percentages
total_count = proficiency_counts.sum()
proficiency_percentages = (proficiency_counts / total_count * 100).round(2)

# Prepare data for plotting
proficiency_data = pd.DataFrame({
    "Proficiency Level": cefr_levels,
    "Count": proficiency_counts.values,
    "Percentage": proficiency_percentages.values
})

# Create a custom color scale for the CEFR levels
color_scale = px.colors.qualitative.Set3  # Or choose another qualitative color scale

# Create a bar plot with a discrete color scale
fig = px.bar(
    proficiency_data,
    x="Proficiency Level",
    y="Percentage",
    title="Distribution of Proficiency Levels (Percentage)",
    labels={"Percentage": "Percentage (%)", "Proficiency Level": "CEFR Levels"},
    text="Percentage",
    color="Proficiency Level",  # Add color differentiation
    color_discrete_sequence=color_scale  # Use a qualitative color scale
)

# Improve layout and aesthetics
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis=dict(categoryorder='array', categoryarray=cefr_levels),
    template="plotly_white",  # Light theme for vibrancy
    title_font=dict(size=20, family='Arial, bold'),
    xaxis_title_font=dict(size=14, family='Arial'),
    yaxis_title_font=dict(size=14, family='Arial'),
)

# Show the plot
fig.show()


## Mapeamento individual

In [33]:
import pandas as pd
import plotly.graph_objects as go


# Load the Excel file
file_path = './Tabela_Transformada_GE_Tales.xlsx'
english_data = pd.read_excel(file_path, sheet_name='Inglês')

# Check if 'Colaborador' column exists
if 'Colaborador' not in english_data.columns:
    raise KeyError("The column 'Colaborador' is not present in the DataFrame")

# Extract relevant columns
cef_columns = english_data.columns[2:]  # Assuming CEFR levels start from the third column
colaboradores = english_data[['Colaborador']]  # Extract "Colaborador" column
dates = english_data[cef_columns]

# Melt the data to long format for Plotly compatibility
long_data = pd.melt(pd.concat([colaboradores, dates], axis=1), id_vars=['Colaborador'], var_name='CEFR Level', value_name='Date')

# Filter out rows with NaN dates
long_data = long_data.dropna(subset=['Date'])

# Convert dates to a consistent format
long_data['Date'] = pd.to_datetime(long_data['Date'], errors='coerce', format='%m/%Y').dt.strftime('%m/%y')

# Create figure
fig = go.Figure()

# Add traces for each individual (Colaborador)
for name in long_data['Colaborador'].unique():
    individual_data = long_data[long_data['Colaborador'] == name]
    fig.add_trace(
        go.Scatter(
            x=individual_data['CEFR Level'],  # X-axis is the CEFR level
            y=[name] * len(individual_data),  # Y-axis is the individual's name
            mode='markers+lines',  # Line connecting milestones
            name=name,
            text=individual_data['Date'],  # Date appears on hover
            marker=dict(size=10),  # Marker size
            line=dict(width=2),  # Line width
        )
    )

# Customize layout
fig.update_layout(
    title='Team Map and Timeline of CEFR Level Progression',
    xaxis_title='CEFR Level',
    yaxis_title='Team Members',
    xaxis=dict(showgrid=True),  # Show gridlines for clarity
    yaxis=dict(showgrid=False),  # Remove horizontal gridlines
    template='plotly_white',  # Clean background
    height=600,  # Chart height
    width=1000,  # Chart width
    legend=dict(title="Team Members"),
)

# Add annotation for dates at the markers
for _, row in long_data.iterrows():
    fig.add_annotation(
        x=row['CEFR Level'],
        y=row['Colaborador'],
        text=row['Date'],
        showarrow=False,
        font=dict(size=10),
        align='center',
        xanchor='center',
    )

# Display the chart
fig.show()
